# FIT5196 Task 1 in Assessment 1
#### Student Name: LICUN LIU
#### Student ID: 30901235

Date: 03/09/2020

Version: 1.0

Environment: Python 2.7.11 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* pandas (for dataframe, included in Anaconda Python 2.7) 
* re (for regular expression, included in Anaconda Python 2.7) 
* numpy (for numpy array, included in Anaconda Python 2.7) 

<font size=3, color="red"> Note: this is a sample notebook only. You will need to fill in the proper markdown and code blocks. You might also want to make necessary changes to the structure to meet your own needs. It is important to make sure the logic of how you finish the assessment is clearly shown in this notebook! </font>

## 1.  Import libraries 

In [1]:
import os
import re
import langid

## 2. Parse XXXX File

In this section, you can write your python scripts to parse the correspondiing file.
You should 
* write proper notes for all code block in this notebook using the Markdown cells
* provide proper comment in your scripts
* run all cells to make sure scripts are runable. If the scripts cannot be run by the assessors, they will not be assessed and zero mark will be given to the task.

In [3]:
path = "./"             # Document Directory
files= os.listdir(path) # Get the name of all documents

# Defined variables
time_list = []
id_list=[]

# Open the output files and write the title in it.
f = open("output.xml", 'a',encoding='utf-8')
f.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n<data>\n")

# Traverse the directory
for file in files: 
    
    # Get the filename
    file_path = path+"/"+file
    (filepath, tempfilename) = os.path.split(file_path)
    (filename,extension) = os.path.splitext(tempfilename)

    # Using filename to extract the time
    name_match = re.match(r'(.*)_(.*).txt', tempfilename)
    if name_match:
        input_file = open(path + "/" + file, 'rb');     # Open the input file
        lines = input_file.read().decode('utf-8')       # Read the file

        time = name_match.group(1)
        id_number = re.findall(r'\"id\"\:\"(?:\d){19}', lines)  # Using regular expression to extract id
        text = re.findall('text\"\:\"(.+?)\"[\,}]', lines)         # Using regular expression to extract text
        
        # Document is in the same date or not 
        if time in time_list:
            pass
        else:
            f.write("</tweets>" + "\n" + "<tweets date=\"" + time + "\">\n")
            time_list.append(time)
        
        # Using loop to read all content
        i = 0
        while i < len(id_number) - 1:
            
            # “id”s are unique and only in English language
            if langid.classify(text[i])[0] == "en" and id_number[i] not in id_list:
                id_list.append(id_number[i])
                f.write("<tweet id=\"" + id_number[i][6:25] + "\">")
                
                # Deal with some emotion icon
                text_emotion = re.findall(r'\\u\w{4}',text[i])
                emotion = lambda text_emotion: text_emotion.encode('ascii').decode('raw_unicode_escape')

                for x in text_emotion:
                    text[i] = text[i].replace(x,emotion(x))

                text[i] = text[i].encode('utf-16', 'surrogatepass').decode('utf-16')
                
                text[i] = text[i].replace('\\n', '\n')      # Deal with '\n' problem
                text[i] = text[i].replace('\'', '&apos;')   # Deal with '‘' problem
                text[i] = text[i].replace('\\\"', '&quot;') # Deal with '\""' problem
                
                f.write(text[i] + "</tweet>\n")
            i = i + 1

f.write("</tweets>\n</data>")
f.close()

## 3. Summary
Give a short summary of your work done above, such as your findings.

#### When I write all of my codes in a loop, it is inconvenient to write the markdown document. So, we need to prevent this situation.